[MongoDB Lecture in Youtube](https://www.youtube.com/watch?v=s44QWegr2l8&list=PLWkguCWKqN9OwcbdYm4nUIXnA2IoXX0LI&index=2)

```
db.<collection>.aggregate([
    <stage1>,
    <stage2>,
    <stage3>,
    
    <stageN>
])
```

#### Server returns a cursor

In [ ]:
show collections
db.persons.aggregate([])

Each stage input documents, output documents <br>
Stages are independent

- Each stage starts from the stage operator
`{$<stageOperator> : {}}``
    - example
        - `{$match : {age : {$gt : 20}}}`
        - `{$group : {_id : "$age"}}`
        - `{$sort : {count : -1}}`

Expression refers to the name of the field in input document
- `"$<fieldname>"`

### Match stage

match uses standard MongoDB queries and supports all query operators

`{$match : {<query>}}`


In [ ]:
db.persons.aggregate([
    {$match : {age: {$gt:25}}}
])

In [ ]:
db.persons.aggregate([
    {$match : {isActive: true}}
])

하단 두 쿼리는 결과가 완전히 동일하다.

In [ ]:
db.persons.aggregate([
    {$match : {tags: {$size : 3}}}
])

db.getCollection('persons').find({tags: {$size : 3}})

### Group Stage

Groups input documents by certain expressions
- `{$group : {_id : <expression>, <field1> : {
    <accumulator1>: ,<expression1>}
    }`

- `
{$group : {_id : "$age"}}
` : Find all distinct of age values. produce new set of document. each document will have just one field

- `{$group : {_id : "$age", gender:"$gender"}}`

In [ ]:
db.persons.aggregate([
    {$group : {_id : "$age"}}
])

**group by nested field**

can use same as above. 

`db.persons.aggregate([
    {$group : {_id : "$company"}}
])
`

`db.persons.aggregate([
    {$group : {_id : "$company.location.country"}}
])
`

`db.persons.aggregate([
    {$group : {_id : "$company.location"}}
])
`

**group by several fields**
- all posible combinations

`db.persons.aggregate([
    { $group : { _id : {age:"$age", gender:"$gender"}}}
])`

`db.persons.aggregate([
    { $group : { _id : {eyeColor:"$eyeColor", favoriteFruit:"$favoriteFruit"}}}
])`

#### `$`match and `$`group

`db.persons.aggregate([
    //stage 1
    {$match : {favoriteFruit: "banana"}}, 
    // stage 2
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}}
])`

`db.persons.aggregate([
    {$match : {gender: "female"}}, 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}}
])`

`db.persons.aggregate([
    {$match : {gender: "female"}}, 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor", gender: "$gender"}}}
])`

아래는 stage순서를 바꾼 상태. 

`db.persons.aggregate([ 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}},
    { $match : {favoriteFruit: "banana"}}
])`

첫번째에서 group을 먼저 하면, 애초에 favoriteFruit라는 컬럼이 없다. fetch 0

아래처럼 하면 결과 나오겠지. 

`
db.persons.aggregate([ 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}},
    { $match : {"_id.age":{$gt:30}}}
])
`

`
db.persons.aggregate([ 
    { $group : { _id : {age:"$age", eyeColor:"$eyeColor"}}},
    { $match : {"_id.eyeColor":"blue"}}
])
`